# The problem

We got tracebacks like that:
```
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib64/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/usr/lib64/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/usr/lib64/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/usr/lib64/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
  File "/home/jhenner/work/miq/http_503_everywhere/.cfme_venv3/lib64/python3.7/site-packages/widgetastic/widget/base.py", line 67, in wrapped
    return method(self, *new_args, **new_kwargs)
  File "/home/jhenner/work/widgetastic.patternfly/src/widgetastic_patternfly/__init__.py", line 348, in __repr__
    return '{}({!r})'.format(type(self).__name__, self.locator)
AttributeError: 'SettingsNavDropdown' object has no attribute 'locator'
Call stack:
  File "/home/jhenner/work/miq/http_503_everywhere/.cfme_venv3/bin/py.test", line 8, in <module>
    sys.exit(main())
  File "/home/jhenner/work/miq/http_503_everywhere/.cfme_venv3/lib64/python3.7/site-packages/_pytest/config/__init__.py", line 125, in main
    config=config
...
  File "/home/jhenner/work/miq/http_503_everywhere/.cfme_venv3/lib64/python3.7/site-packages/widgetastic/browser.py", line 438, in is_displayed
    return self.move_to_element(locator, *args, **kwargs).is_displayed()
  File "/home/jhenner/work/miq/http_503_everywhere/.cfme_venv3/lib64/python3.7/site-packages/widgetastic/utils.py", line 679, in wrap
    return method(*args, **kwargs)
  File "/home/jhenner/work/miq/http_503_everywhere/.cfme_venv3/lib64/python3.7/site-packages/widgetastic/browser.py", line 456, in move_to_element
    self.logger.debug
    ('move_to_element: %r', locator)
Unable to print the message and arguments - possible formatting error.
Use the traceback above to help find the error.
```

# What is wrong?
Well it takes some reading effort but the important is:
```AttributeError: 'SettingsNavDropdown' object has no attribute 'locator'```

It is a descendant of `widgetastic_patternfly.NavDropdown`. It has no `__repr__` defined, so the `NavDropdown` is what we should look at.

In [4]:
from widgetastic_patternfly import NavDropdown
str(NavDropdown())

'NavDropdown()'

Well it works just fine...

In [291]:
nav = NavDropdown()
nav.locator

AttributeError: 'WidgetDescriptor' object has no attribute 'locator'

In [290]:
NavDropdown().__repr__

<bound method WidgetDescriptor.__repr__ of NavDropdown()>

Hmm, no locator... but wait! A `WidgetDescriptor`? I've just created `NavDropDown`, dind't I ?

In [292]:
type(NavDropdown())

widgetastic.widget.base.WidgetDescriptor

In [293]:
NavDropdown??

* nothing suspiccious in the source of `NavDropdown`
* With more reading you see there is some **Metaclasses** magic being done.

In [243]:
from widgetastic_patternfly import Widget
Widget??

```python
Init signature: Widget(*args, **kwargs)
Source:        
class Widget(object, metaclass=WidgetMetaclass):
    """Base class for all UI objects.

    Does couple of things:

        * Ensures it gets instantiated with a browser or another widget as parent. If you create an
          instance in a class, it then creates a :py:class:`WidgetDescriptor` which is then invoked
          on the instance and instantiates the widget with underlying browser.
        * Implements some basic interface for all widgets.
        
    If you are inheriting from this class, you **MUST ALWAYS** ensure that the inherited class
    has an init that always takes the ``parent`` as the first argument. You can do that on your
    own, setting the parent as ``self.parent`` or you can do something like this:        
    
   .. code-block:: python

        def __init__(self, parent, arg1, arg2, logger=None):
            super(MyClass, self).__init__(parent, logger=logger)
            # or if you have somehow complex inheritance ...
            Widget.__init__(self, parent, logger=logger)

```

# The magic of widgetastic

In [297]:
from widgetastic_patternfly import NavDropdown, View

class MyView(View):
    widget = NavDropdown()

type(MyView.widget)

widgetastic.widget.base.WidgetDescriptor

After hours or more of figguring out:
 * **Perhaps** I need to instantiate the View
 * I need a `Browser` intance to do it.
 * there were some problems to do it, but today I new I can just use `MagicMock`.

In [301]:
from widgetastic.browser import Browser
from unittest.mock import MagicMock

selenium_mock = MagicMock()
view = MyView(Browser(selenium_mock))
type(view.widget)

widgetastic_patternfly.NavDropdown

**Looks promissing!** We sucessfully created a real `NavDropdown`. Let's try it fails:

In [271]:
view.widget

AttributeError: 'NavDropdown' object has no attribute 'locator'

**OMG! OMG! OMG! OMG! It does Fail**. Let's fix it using some mokey-patching.

In [284]:
def fixed_repr(self):
    return 'The fixed NavDropdown'
NavDropdown.__repr__ = fixed_repr

class MyView(View):
    widget = NavDropdown()

view = MyView(Browser(MagicMock()))
view.widget

The fixed NavDropdown

Ok. We have the fix, but there is **lots of `Widget` descendants** defined in the wrapanapi. The wrapanapi is **quite intricated** pile of **Metaclass** magic **Descriptors** and dynamic code smells like `isinstance` and `hassattr`. I should make a test the __str__ and __repr__ of all the classes it exports. But how?

```python
from widgetastic_patternfly import View, Button, Kebab, Text, ...
def test_str(browser):
    class TestView(View):
        any_button = Button()  # will pick up first button...
        button1 = Button('Default Normal')
        button2 = Button(title='Destructive title')
        button3 = Button(title='noText', classes=[Button.PRIMARY])
        kebab = Kebab(...)
        kebab2 = Kebab(id="")
        ...
    view = TestView(browser)
    for widget in view.any_button, view.button1, view.button2, view.button3, ...:
        assert type(str(widget)) is str
```

Not great heh... mainly there is **no guarantee that newly defined class will get tested.**


# Creating the unit-test testing all the exported components

Well I learned something about what happens **behind the seens** of Python types and classes.

In [323]:
DynamicType = type('TheTestView', (View,), dict(some_attribute="attribute value"))

**We just created a type in dynamic way.**

In [324]:
class OrdinaryType(View):
    some_attribute = "attribute_value"

DynamicType.__module__, OrdinaryType.__module__

('widgetastic.widget.base', '__main__')

This is just a small difference of the behavior. that may bite you some day. Otherwise the results are **identical**

In [327]:
import widgetastic_patternfly

import inspect
inspect.getmembers(widgetastic_patternfly)

[('AboutModal', widgetastic_patternfly.AboutModal),
 ('Accordion', widgetastic_patternfly.Accordion),
 ('AggregateStatusCard', widgetastic_patternfly.AggregateStatusCard),
 ('AggregateStatusMiniCard', widgetastic_patternfly.AggregateStatusMiniCard),
 ('BarChart', widgetastic_patternfly.BarChart),
 ('BaseInput', widgetastic.widget.input.BaseInput),
 ('BootstrapNav', widgetastic_patternfly.BootstrapNav),
 ('BootstrapSelect', widgetastic_patternfly.BootstrapSelect),
 ('BootstrapSwitch', widgetastic_patternfly.BootstrapSwitch),
 ('BootstrapTreeview', widgetastic_patternfly.BootstrapTreeview),
 ('BreadCrumb', widgetastic_patternfly.BreadCrumb),
 ('Button', widgetastic_patternfly.Button),
 ('CandidateNotFound', widgetastic_patternfly.CandidateNotFound),
 ('CheckableBootstrapTreeview',
  widgetastic_patternfly.CheckableBootstrapTreeview),
 ('ClickableMixin', widgetastic.widget.base.ClickableMixin),
 ('DatePicker', widgetastic_patternfly.DatePicker),
 ('Dropdown', widgetastic_patternfly.Dropdo

In [326]:
from IPython.lib.pretty import pprint

class WidgetDescriptor:
    def __new__(self, *args, **kwargs):
        pprint(locals())
        return "Any random object"
    

class MyView:
    the_attribute = WidgetDescriptor("arg", foo="kwarg")
    
MyView.the_attribute, MyView().the_attribute

{'self': __main__.WidgetDescriptor,
 'args': ('arg',),
 'kwargs': {'foo': 'kwarg'}}


('Any random object', 'Any random object')

In [241]:
from IPython.lib.pretty import pprint

class Metaclass(type): 
    def __new__(cls, name, bases, attrs):
        """
        Crates new type
        """
        print(f"======Metaclass.new is creating a type named {name}")
        pprint(locals())

    def __call__(self, *args, **kwargs):
        """
        Instantiates the class to an object
        """
        pprint(locals())
        print(f"======Metaclass.call is creating an instance of {self}")
        obj = super().__call__(*args, **kwargs)
        assert isinstance(obj, object)
        return obj

    def __init__(self, *args, **kwargs):
        """
        Fills the __dict__
        """
        print(f"=====Metaclass.init lets the {self} modify itself.")
        pprint(locals())
        
        
class WidgetDescriptor:
    """This class handles instantiating and caching of the widgets on view.

    It stores the class and the parameters it should be instantiated with. Once it is accessed from
    the instance of the class where it was defined on, it passes the instance to the widget class
    followed by args and then kwargs.

    It also acts as a counter, so you can then order the widgets by their "creation" stamp.
    """
    def __init__(self, klass, *args, **kwargs):
        self.klass = klass
        self.args = args
        self.kwargs = kwargs
        
        
class Widget(metaclass=Metaclass):
    foo = Widget()
    
Widget.container

{'self': __main__.Widget,
 'args': (),
 'kwargs': {},
 '__class__': __main__.Metaclass}
======Metaclass.call is creating an instance of <class '__main__.Widget'>
======Metaclass.new is creating a type named Widget
{'cls': __main__.Metaclass,
 'name': 'Widget',
 'bases': (),
 'attrs': {'__module__': '__main__',
  '__qualname__': 'Widget',
  'foo': <__main__.Widget at 0x7f0926034b50>},
 '__class__': __main__.Metaclass}
=====Metaclass.init lets the <class '__main__.Widget'> modify itself.
{'self': __main__.Widget,
 'args': ('Widget',
  (),
  {'__module__': '__main__',
   '__qualname__': 'Widget',
   'foo': <__main__.Widget at 0x7f0926034b50>,
   'container': 'Widget'}),
 'kwargs': {}}


'Widget'

In [142]:
class View(Widget):
    attribute = "content"

======Metaclass.new is creating a type named View
{'cls': __main__.Metaclass,
 'name': 'View',
 'bases': (__main__.Widget,),
 'attrs': {'__module__': '__main__',
  '__qualname__': 'View',
  'attribute': 'content'},
 '__class__': __main__.Metaclass}
=====Metaclass.init lets the <class '__main__.View'> modify itself.
{'self': __main__.View,
 'args': ('View',
  (__main__.Widget,),
  {'__module__': '__main__', '__qualname__': 'View', 'attribute': 'content'}),
 'kwargs': {}}


In [164]:
class LazyType:
    def __new__(cls, *args, **kwargs):
        container = Widget
        if (args and isinstance(args[0], container)) \
                or (isinstance(kwargs.get('parent', None), container)):
            return super().__new__(cls)
        else:
            return WidgetDescriptor(cls, *args, **kwargs)

In [166]:
LazyType(), LazyType(Widget())

{'self': __main__.WidgetDescriptor, 'args': (__main__.LazyType,), 'kwargs': {}}
{'self': __main__.Widget,
 'args': (),
 'kwargs': {},
 '__class__': __main__.Metaclass}
======Metaclass.call is creating an instance of <class '__main__.Widget'>


('Any random object', <__main__.LazyType at 0x7f0926037c90>)

In [174]:
class Widget:
    lazy_attr = LazyType()
    not_so_lazy_attr = LazyType(Widget())
    
Widget.lazy_attr, Widget.not_so_lazy_attr

{'self': __main__.WidgetDescriptor, 'args': (__main__.LazyType,), 'kwargs': {}}


('Any random object', <__main__.LazyType at 0x7f0925ea3050>)

In [29]:
from widgetastic_patternfly import View, Widget
from IPython.lib.pretty import pprint
import inspect


class WidgetDescriptor:
    def __init__(self, klass, *args, **kwargs):
        self.klass = klass
        self.args = args
        self.kwargs = kwargs
        
    def __get__(self, owner, owner_class):
        if obj is None:  # class access
            return self
        else:
            new_object = self.klass(obj, *args, **kwargs)
            return new_object

class Meta(type):
    def __new__(cls, name, bases, attrs):
        new_attrs={}
        for key, value in attrs.items():
            if inspect.isclass(value) and issubclass(value, View):
                new_attrs[key] = WidgetDescriptor(value)

        return super(Meta, cls).__new__(cls, name, bases, new_attrs)

class MyWidget(metaclass=Meta):
    def __new__(self, *args, **kwargs):
        if (args and isinstance(args[0], (Widget))) \
                or ('parent' in kwargs and isinstance(kwargs['parent'], (Widget))):
            return super().__new__(cls)
        else:
            return WidgetDescriptor(cls, *args, **kwargs)
        
        
class MyView:
    the_attribute = WidgetDescriptor()
    
MyView.the_attribute
MyView().the_attribute

/home/jhenner/work/types_talk/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: __class__ not set defining 'MyWidget' as <class '__main__.MyWidget'>. Was __classcell__ propagated to type.__new__?


TypeError: __init__() missing 1 required positional argument: 'klass'